## Introduction

In this example, we train a model to learn to add two numbers, provided as strings.

**Example:**

- Input: "535+61"
- Output: "596"

Input may optionally be reversed, which was shown to increase performance in many tasks
 in: [Learning to Execute](http://arxiv.org/abs/1410.4615) and
[Sequence to Sequence Learning with Neural Networks](http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf).

Theoretically, sequence order inversion introduces shorter term dependencies between
 source and target for this problem.

**Results:**

For two digits (reversed):

+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits (reversed):

+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits (reversed):

+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits (reversed):

+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

14 digits (reversed):

+ Three layers LSTM (128 HN), 700k training examples = 99% train/test accuracy in 25 epochs



## Setup

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 1000000
DIGITS = 30
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

## Generate the data

In [2]:

class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

Generating data...
Total questions: 1000000


## Vectorize the data

In [3]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Vectorization...


<ipython-input-3-195dd5dc8a21>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
<ipython-input-3-195dd5dc8a21>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)


Training Data:
(900000, 61, 12)
(900000, 31, 12)
Validation Data:
(100000, 61, 12)
(100000, 31, 12)


## Build the model

In [4]:
print("Build model...")
num_layers = 3  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 31, 128)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 31, 128)           131584    
                                                                 
 lstm_2 (LSTM)               (None, 31, 128)           131584    
                                                                 
 lstm_3 (LSTM)               (None, 31, 128)           131584    
                                                                 
 dense (Dense)               (None, 31, 12)            1548      
                                         

## Train the model

In [5]:
from tqdm.auto import trange
from tqdm.keras import TqdmCallback

epochs = 25
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in trange(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        callbacks=[TqdmCallback(verbose=1)],
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)

  0%|          | 0/24 [00:00<?, ?it/s]


Iteration 1


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

1/1 [==============================] - 1s 1s/step
Q 4780942590822937+78580104305488534742435                      T 78580109086431125565372         ☒ 78580111105555777777778        
1/1 [==============================] - 0s 22ms/step
Q 157738985352689+81558486650109181604392                       T 81558486807848166957081         ☒ 81588486833355777777771        
1/1 [==============================] - 0s 24ms/step
Q 409435569153142766655897982+30449418611                       T 409435569153142797105316593     ☒ 409635579177777777777777778    
1/1 [==============================] - 0s 22ms/step
Q 209276797222218041741125051+0                                 T 209276797222218041741125051     ☒ 209354597222210000000777770    
1/1 [==============================] - 0s 21ms/step
Q 96519872072286627+97862010997867860174                        T 97958530869940146801            ☒ 96231175555557777773           
1/1 [==============================] - 0s 24ms/step
Q 43434498406998826026002+6748

...

0batch [00:00, ?batch/s]

1/1 [==============================] - 0s 21ms/step
Q 422375698915229+25933164179607447                             T 26355539878522676               ☒ 26354655555555665              
1/1 [==============================] - 0s 24ms/step
Q 2929846196+27666438135287624423071856                         T 27666438135287627352918052      ☒ 27666438135287627340006666     
1/1 [==============================] - 0s 22ms/step
Q 17133605263824574322616+151297149772034                       T 17133605415121724094650         ☒ 17133605415155555666666        
1/1 [==============================] - 0s 22ms/step
Q 69617951604963448130091030+1078610225451020751449860          T 70696561830414468881540890      ☒ 70696566666666666666666660     
1/1 [==============================] - 0s 19ms/step
Q 79608677350+2211833810021774470109                            T 2211833810101383147459          ☒ 2211833810101350666660         
1/1 [==============================] - 0s 24ms/step
Q 974526964109898662886738+2

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

1/1 [==============================] - 0s 23ms/step
Q 441+329611800                                                 T 329612241                       ☒ 329612240                      
1/1 [==============================] - 0s 25ms/step
Q 472641389199273202435946803+53149                             T 472641389199273202435999952     ☒ 472641389199273202435099957    
1/1 [==============================] - 0s 20ms/step
Q 7446198591972163065023150077+3370991396394                    T 7446198591972166436014546471    ☒ 7446198591972166436611111111   
1/1 [==============================] - 0s 21ms/step
Q 80744254740400076842828357+3                                  T 80744254740400076842828360      ☑ 80744254740400076842828360     
1/1 [==============================] - 0s 25ms/step
Q 488153713169075+72581927186584783                             T 73070080899753858               ☒ 73070111111111111              
1/1 [==============================] - 0s 23ms/step
Q 583317301857011489902382+8

You'll get to 99+% validation accuracy after ~30 epochs.

Example available on HuggingFace.

| Trained Model | Demo |
| :--: | :--: |
| [![Generic badge](https://img.shields.io/badge/🤗%20Model-Addition%20LSTM-black.svg)](https://huggingface.co/keras-io/addition-lstm) | [![Generic badge](https://img.shields.io/badge/🤗%20Spaces-Addition%20LSTM-black.svg)](https://huggingface.co/spaces/keras-io/addition-lstm) |

In [6]:
model.save_weights('rnn_for_addition')

In [7]:
model.save("model.keras")